## Predicting the Sale Price of Bulldozers (Kaggle Competition)

## 1. Problem Definition

The goal of this project is to predict the sale price of bulldozers at auction.  
Predictions are based on usage, equipment type, and configuaration.  
The data is sourced from auction result postings.  
Type of machine learning problem: **supervised learning / time series regression**

## 2. Data

Data is downloaded from the *Bluebook for Bulldozers* past Kaggle competition:  
[Bluebook for Bulldozers Kaggle Competition](https://www.kaggle.com/c/bluebook-for-bulldozers/overview)  
There are three main datasets:  
* **Train.csv** is the training set, which contains data through the end of 2011.
* **Valid.csv** is the validation set, which contains data from January 1, 2012 - April 30, 2012.  
The score on this set was used to create the public leaderboard.
* **Test.csv** is the test set, which contains data from May 1, 2012 - November 30, 2012.  
The score on this set determined the final rank for the competition.

## 3. Evaluation

The competition evaluation metric was *root mean squared log error (RMSLE)*.  
**Project goal:** To minimize the difference between actual and predicted prices, i.e., to minimize RMSLE.

## 4. Features

Kaggle provided a data dictiorany for these datasets.  
See `data-dictionary.xlsx` in the project folder.

#### Importing the tools

In [ ]:
### imports
import numpy, sklearn
from pandas import read_csv, DataFrame
from matplotlib import pyplot

#### Importing data

**Parsing dates**

Working with time series data requires the date/time information to be parsed into datetime format.  
Date/time columns are parsed using the `parse_dates=` parameter of `read_csv()`.

In [ ]:
### importing bulldozer dataset from file
bulldozer_df: DataFrame = read_csv(
    filepath_or_buffer="../Large-Files/train-valid.csv",
    parse_dates=["saledate"],
    low_memory=False)

#### Verifying dataframe integrity

In [ ]:
### displaying dataframe dimension, column name, missing value, and datatype info
bulldozer_df.info()

In [ ]:
### checking for missing values
bulldozer_df.isna().sum()

In [ ]:
bulldozer_data["SalePrice"].plot.hist();